In [ ]:
from torch import nn
import torch

# Setting the device to use

# Default device is CPU
device = "cpu"

# If CUDA is available, use it
if torch.cuda.is_available():
    device = "cuda"
# If running on macOS and with Metal, use it
elif torch.backends.mps.is_available():
    device ="mps"

print("Using device: ", device)
torch.device(device)

: 